## Imports

In [1]:
%pylab inline
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import pickle
import seaborn as sns
from datetime import datetime

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
#https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html
from statsmodels.tsa.arima_model import ARIMA

## Data loading & cleaning

In [3]:
from helper import clean_data
from helper import vix_prime_combine

In [4]:
pct_df, vix_close , prime, weekly_vix, monthly_vix = clean_data()

In [5]:
from helper import split_data

In [6]:
df_len = pct_df.shape[0]
train = pct_df.iloc[:df_len-200,0]
train = train.values
validation = pct_df.iloc[df_len-200:df_len-100,0]
validation = validation.values
test = pct_df.iloc[df_len-100:,0]

In [7]:
model = ARIMA(train, order=(1,0,0)).fit()

/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `ar

In [8]:
def RMSE(validation_points, prediction_points):
   """
   Calculate RMSE between two vectors
   """
   x = np.array(validation_points)
   y = np.array(prediction_points)

   return np.sqrt(np.mean((x - y)**2))


In [9]:
def test_model(df,p,q):
    preds = []
    df = list(df)

    for i in validation:
        model = ARIMA(df, order=(p,0,q)).fit()
        pred = model.forecast()[0][0]
        preds.append(pred)
        df.append(i)

    #plt.plot(validation)
    #plt.plot(preds)
    
    return RMSE(validation,np.array(preds))


In [10]:
test_model(train,1,0)

0.0378392338537955

In [ ]:
res= []
for i in range(1,5):
    for q in range(1,2):
        model = ARIMA(train,order=(i,0,q)).fit().forecast(100)
        res.append([i,q,RMSE(validation,model[0])])
        
res

These all do worse. Best is 7,0,1 at RMSE = 0.04635423458910249

In [72]:
monthly_vix = pct_df.resample('M').sum()